# Interview task

In [121]:
RANDOM_STATE = 20

## check GPU

In [122]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Mar 21 13:54:58 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:CD:00.0 Off |                    0 |
|  0%   37C    P0    77W / 300W |   2513MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [123]:
# !conda install conda-forge::pytorch-gpu

In [124]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda:0


## Check dataset

In [125]:
import pandas as pd

In [126]:
df = pd.read_csv('substance_interactions.csv')

In [127]:
df.head()

,PREDICATION_ID,PMID,PREDICATE,INDICATOR_TYPE,PREDICATE_START_INDEX,PREDICATE_END_INDEX,SUBJECT_TEXT,SUBJECT_SEMTYPE,SUBJECT_START_INDEX,SUBJECT_END_INDEX,...,OBJECT_START_INDEX,OBJECT_END_INDEX,OBJECT_SCORE,OBJECT_DIST,OBJECT_MAXDIST,OBJECT_CUI,OBJECT_NOVELTY,TYPE,SENTENCE,LABEL
0,P3100,6499897,INTERACTS_WITH,NOM,1298,1304,SA,orch,1235,1237,...,1329,1332,1000,2,2,C0004057,1,ab,"Nor did administration of SA, diflunisal or AS...",n
1,P3101,8369307,INHIBITS,VERB,890,899,rHF,aapp,785,788,...,912,919,888,1,15,C0242417,1,ab,A comparative study of recombinant L-cha...,n
2,P3102,3711333,INHIBITS,VERB,1527,1534,alkaloids,orch,1508,1517,...,1541,1550,1000,1,1,C0003805,1,ab,These findings suggest that some nicotinic alk...,y
3,P3103,11742534,INTERACTS_WITH,NOM,746,753,amino acids,aapp,703,714,...,741,745,694,0,4,C0169658|3716,1,ab,With a truncated chimaeric IL-5Rbeta-gp1...,y
4,P3104,244385,STIMULATES,ADJ,410,419,Neutral endopeptidase,aapp,374,401,...,480,491,1000,3,5,C0039815,1,ab,"Neutral endopeptidase, a zinc-dependent ...",n


In [128]:
df.columns

Index(['PREDICATION_ID', 'PMID', 'PREDICATE', 'INDICATOR_TYPE',
       'PREDICATE_START_INDEX', 'PREDICATE_END_INDEX', 'SUBJECT_TEXT',
       'SUBJECT_SEMTYPE', 'SUBJECT_START_INDEX', 'SUBJECT_END_INDEX',
       'SUBJECT_SCORE', 'SUBJECT_DIST', 'SUBJECT_MAXDIST', 'SUBJECT_CUI',
       'SUBJECT_NOVELTY', 'OBJECT_TEXT', 'OBJECT_SEMTYPE',
       'OBJECT_START_INDEX', 'OBJECT_END_INDEX', 'OBJECT_SCORE', 'OBJECT_DIST',
       'OBJECT_MAXDIST', 'OBJECT_CUI', 'OBJECT_NOVELTY', 'TYPE', 'SENTENCE',
       'LABEL'],
      dtype='object')

In [129]:
def pre_processing(example):
    sentence = example['SENTENCE']
    subject = example['SUBJECT_TEXT']
    object = example['OBJECT_TEXT']
    relation = example['PREDICATE']
    # text = f"{subject} [SEP] {relation} [SEP] {object} [SEP] {sentence}"
    text = f"{sentence} [SEP] {subject} , {relation} , {object}"
    return text

df['triple_with_sentence'] = df.apply(pre_processing,axis=1)
df.head()

,PREDICATION_ID,PMID,PREDICATE,INDICATOR_TYPE,PREDICATE_START_INDEX,PREDICATE_END_INDEX,SUBJECT_TEXT,SUBJECT_SEMTYPE,SUBJECT_START_INDEX,SUBJECT_END_INDEX,...,OBJECT_END_INDEX,OBJECT_SCORE,OBJECT_DIST,OBJECT_MAXDIST,OBJECT_CUI,OBJECT_NOVELTY,TYPE,SENTENCE,LABEL,triple_with_sentence
0,P3100,6499897,INTERACTS_WITH,NOM,1298,1304,SA,orch,1235,1237,...,1332,1000,2,2,C0004057,1,ab,"Nor did administration of SA, diflunisal or AS...",n,"Nor did administration of SA, diflunisal or AS..."
1,P3101,8369307,INHIBITS,VERB,890,899,rHF,aapp,785,788,...,919,888,1,15,C0242417,1,ab,A comparative study of recombinant L-cha...,n,A comparative study of recombinant L-cha...
2,P3102,3711333,INHIBITS,VERB,1527,1534,alkaloids,orch,1508,1517,...,1550,1000,1,1,C0003805,1,ab,These findings suggest that some nicotinic alk...,y,These findings suggest that some nicotinic alk...
3,P3103,11742534,INTERACTS_WITH,NOM,746,753,amino acids,aapp,703,714,...,745,694,0,4,C0169658|3716,1,ab,With a truncated chimaeric IL-5Rbeta-gp1...,y,With a truncated chimaeric IL-5Rbeta-gp1...
4,P3104,244385,STIMULATES,ADJ,410,419,Neutral endopeptidase,aapp,374,401,...,491,1000,3,5,C0039815,1,ab,"Neutral endopeptidase, a zinc-dependent ...",n,"Neutral endopeptidase, a zinc-dependent ..."


## Tokenizer

In [130]:
# !pip install git+https://github.com/huggingface/transformers

In [131]:
# MODEL_NAME = "enoch/llama-65b-hf"

from transformers import AutoTokenizer, LlamaForSequenceClassification, LlamaTokenizer, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/tiny-random-LlamaForSeqClass")

# from transformers import LlamaForSequenceClassification, LlamaTokenizer


# tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


#### test tokenizer for a triple with corresponding sentence

In [132]:
example = df['triple_with_sentence'].iloc[0]
example

'Nor did administration of SA, diflunisal or ASA itself impair the       anti-aggregatory effect of a fresh test dose of ASA. [SEP] SA , INTERACTS_WITH , ASA'

In [133]:
tokenizer(example, return_tensors='pt', truncation=True)

{'input_ids': tensor([[    1,  4186,  1258, 17517,   310, 16698, 29892,   958, 29880,   348,
           275,   284,   470,  3339, 29909,  3528,  2411,  1466,   278,   539,
          9418, 29899, 26193,  7606,  2779,   310,   263, 10849,  1243,   437,
           344,   310,  3339, 29909, 29889,   518,  1660, 29925, 29962, 16698,
          1919,  2672,  4945, 17923, 29903, 29918, 29956, 13054,  1919,  3339,
         29909]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}

In [134]:
def processing(example):
    res = tokenizer(example['triple_with_sentence'])
    # res['label'] = example['LABEL']
    res['label'] = 1 if example['LABEL']=='y' else 0
    return res
df['data'] = df.apply(processing, axis=1)

In [135]:
df['data'][0]

{'input_ids': [1, 4186, 1258, 17517, 310, 16698, 29892, 958, 29880, 348, 275, 284, 470, 3339, 29909, 3528, 2411, 1466, 278, 539, 9418, 29899, 26193, 7606, 2779, 310, 263, 10849, 1243, 437, 344, 310, 3339, 29909, 29889, 518, 1660, 29925, 29962, 16698, 1919, 2672, 4945, 17923, 29903, 29918, 29956, 13054, 1919, 3339, 29909], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': 0}

In [136]:
df.columns

Index(['PREDICATION_ID', 'PMID', 'PREDICATE', 'INDICATOR_TYPE',
       'PREDICATE_START_INDEX', 'PREDICATE_END_INDEX', 'SUBJECT_TEXT',
       'SUBJECT_SEMTYPE', 'SUBJECT_START_INDEX', 'SUBJECT_END_INDEX',
       'SUBJECT_SCORE', 'SUBJECT_DIST', 'SUBJECT_MAXDIST', 'SUBJECT_CUI',
       'SUBJECT_NOVELTY', 'OBJECT_TEXT', 'OBJECT_SEMTYPE',
       'OBJECT_START_INDEX', 'OBJECT_END_INDEX', 'OBJECT_SCORE', 'OBJECT_DIST',
       'OBJECT_MAXDIST', 'OBJECT_CUI', 'OBJECT_NOVELTY', 'TYPE', 'SENTENCE',
       'LABEL', 'triple_with_sentence', 'data'],
      dtype='object')

## split the data, training set 70%, validation set 15%, test set 15%

In [137]:
from sklearn.model_selection import train_test_split


train_data, test_data = train_test_split(df, test_size=0.3, random_state=RANDOM_STATE)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=RANDOM_STATE)

In [138]:
print(len(train_data),len(val_data),len(test_data))

2100 450 450


In [139]:
train_data = train_data.reset_index()
val_data = val_data.reset_index()
test_data = test_data.reset_index()

In [140]:
from transformers import DataCollatorWithPadding
# import evaluate

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# accuracy = evaluate.load('accuracy')

In [141]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def compute_metrics(eval_pred):
	    predictions, labels = eval_pred
	    predictions = np.argmax(predictions, axis=1)
	    
	    # Calculate precision, recall, and F1 score
	    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
	    
	    return {
	        'accuracy': accuracy_score(labels, predictions),
	        'precision': precision,
	        'recall': recall,
	        'f1': f1
	    }

## BERT model

In [142]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification


labels = ['n', 'y']
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}

print('id2label:', id2label)
print('label2id:', label2id)

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=len(labels), id2label=id2label, label2id=label2id)
# model = DistributedLlamaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels), id2label=id2label, label2id=label2id)
# model = LlamaForSequenceClassification.from_pretrained("llama-7b", num_labels=len(labels), id2label=id2label, label2id=label2id)
model = AutoModelForSequenceClassification.from_pretrained("HuggingFaceH4/tiny-random-LlamaForSeqClass")


model.config.pad_token_id = tokenizer.pad_token_id

# Only train last classifier layer
# for param in model.base_model.parameters():
#     param.requires_grad = False
# # resize model embedding to match new tokenizer
# model.resize_token_embeddings(len(tokenizer))

# # fix model padding token id
# model.config.pad_token_id = model.config.eos_token_id

id2label: {0: 'n', 1: 'y'}
label2id: {'n': 0, 'y': 1}


In [143]:
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 16, padding_idx=31999)
    (layers): ModuleList(
      (0-1): 2 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=16, out_features=16, bias=False)
          (k_proj): Linear(in_features=16, out_features=16, bias=False)
          (v_proj): Linear(in_features=16, out_features=16, bias=False)
          (o_proj): Linear(in_features=16, out_features=16, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=16, out_features=64, bias=False)
          (up_proj): Linear(in_features=16, out_features=64, bias=False)
          (down_proj): Linear(in_features=64, out_features=16, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (score): Lin

In [144]:
# Freeze all layers except the last one
for param in model.base_model.parameters():
    param.requires_grad = False

# for name, param in model.named_parameters():
#     print(name)
    
for name, param in model.named_parameters():
    if name.startswith('model.norm'): #name.startswith('model.layers.1') or 
        param.requires_grad = True


for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


##  Training

In [145]:
training_args = TrainingArguments(
    output_dir='my_best_model',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

In [146]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data['data'],
    eval_dataset=val_data['data'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

/home/csci5541/zhan8023/.conda/envs/pytorch_jupyter_a40/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.690866,0.522222,0.583893,0.362500,0.447301
2,No log,0.690671,0.537778,0.588889,0.441667,0.504762
3,No log,0.690533,0.535556,0.583784,0.450000,0.508235
4,No log,0.690404,0.542222,0.591398,0.458333,0.516432
5,No log,0.690297,0.551111,0.601064,0.470833,0.528037
6,No log,0.690208,0.551111,0.601064,0.470833,0.528037
7,No log,0.690134,0.551111,0.601064,0.470833,0.528037
8,0.692500,0.690087,0.551111,0.600000,0.475000,0.530233
9,0.692500,0.690058,0.551111,0.600000,0.475000,0.530233
10,0.692500,0.690049,0.551111,0.600000,0.475000,0.530233


Checkpoint destination directory my_best_model/checkpoint-66 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_best_model/checkpoint-132 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_best_model/checkpoint-198 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_best_model/checkpoint-264 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_best_model/checkpoint-330 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_best_model/checkpoint-396 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_best_model/checkpoint-462 already exists and is non-empty. Saving will 

TrainOutput(global_step=660, training_loss=0.692364097364021, metrics={'train_runtime': 19.5122, 'train_samples_per_second': 1076.251, 'train_steps_per_second': 33.825, 'total_flos': 175098276864.0, 'train_loss': 0.692364097364021, 'epoch': 10.0})

In [147]:
trainer.evaluate(train_data['data'])

{'eval_loss': 0.6918833255767822,
 'eval_accuracy': 0.5347619047619048,
 'eval_precision': 0.5900116144018583,
 'eval_recall': 0.44876325088339225,
 'eval_f1': 0.5097842448569995,
 'eval_runtime': 0.7132,
 'eval_samples_per_second': 2944.351,
 'eval_steps_per_second': 92.537,
 'epoch': 10.0}

In [148]:
trainer.evaluate(val_data['data'])

{'eval_loss': 0.6900485754013062,
 'eval_accuracy': 0.5511111111111111,
 'eval_precision': 0.6,
 'eval_recall': 0.475,
 'eval_f1': 0.5302325581395348,
 'eval_runtime': 0.1694,
 'eval_samples_per_second': 2656.827,
 'eval_steps_per_second': 88.561,
 'epoch': 10.0}

In [149]:
trainer.evaluate(test_data['data'])

{'eval_loss': 0.6920243501663208,
 'eval_accuracy': 0.5355555555555556,
 'eval_precision': 0.5631578947368421,
 'eval_recall': 0.4592274678111588,
 'eval_f1': 0.5059101654846335,
 'eval_runtime': 0.166,
 'eval_samples_per_second': 2711.46,
 'eval_steps_per_second': 90.382,
 'epoch': 10.0}